<a href="https://colab.research.google.com/github/Czar210/PUC-CDIA/blob/3-semestre/Rooneyentrega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
!pip install ortools
from ortools.linear_solver import pywraplp
Comus='/content/za.xlsx'
data = pd.read_csv('/content/tabela_necessidades_dieteticas.csv')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 30.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.4.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.2

In [3]:
data

,Sexo,Peso (kg),Proteínas (g/dia),Carboidratos (g/dia),Gorduras (g/dia),Vitamina A (mcg/dia),Vitamina C (mg/dia),Cálcio (mg/dia),Ferro (mg/dia)
0,Homem,50,40,200,50,900,90,1000,8
1,Homem,60,48,240,60,900,90,1000,8
2,Homem,70,56,280,70,900,90,1000,8
3,Homem,80,64,320,80,900,90,1000,8
4,Homem,90,72,360,90,900,90,1000,8
5,Homem,100,80,400,100,900,90,1000,8
6,Mulher,50,40,200,50,700,75,1000,18
7,Mulher,55,43,215,55,700,75,1000,18
8,Mulher,60,46,230,60,700,75,1000,18
9,Mulher,65,49,245,65,700,75,1000,18


In [4]:
a = 'Mulher'
b = 76

In [5]:
from sklearn.linear_model import LinearRegression

# Definir as variáveis de entrada e as variáveis-alvo
input_features = ["Peso (kg)"]
target_features = ["Proteínas (g/dia)", "Carboidratos (g/dia)", "Gorduras (g/dia)", "Vitamina A (mcg/dia)", "Vitamina C (mg/dia)", "Cálcio (mg/dia)", "Ferro (mg/dia)"]

# Entrada: Sexo e peso


sexo = a
peso = b

# Filtrar dados por sexo
filtered_data = data[data['Sexo'] == sexo]

# Treinar o modelo de regressão linear para cada variável-alvo
models = {}
for target in target_features:
    model = LinearRegression()
    model.fit(filtered_data[input_features], filtered_data[target], sample_weight=None)
    models[target] = model

# Prever os valores intermediários para o peso dado
predictions = {}
for target, model in models.items():
    prediction = model.predict(pd.DataFrame(data=[[peso]], columns=input_features))
    predictions[target] = prediction[0]
lista = []
# Imprimir os resultados
print(f"Valores interpolados para {sexo} com {peso} kg:")
for target, value in predictions.items():
    print(f"{target}: {value:.2f}")
    lista.append(value)




Valores interpolados para Mulher com 76 kg:
Proteínas (g/dia): 55.60
Carboidratos (g/dia): 278.00
Gorduras (g/dia): 76.00
Vitamina A (mcg/dia): 700.00
Vitamina C (mg/dia): 75.00
Cálcio (mg/dia): 1000.00
Ferro (mg/dia): 18.00


In [6]:
#limpeza de alguns valores pelo python
df = pd.read_excel('/content/za.xlsx')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.fillna(0, inplace=True)

alimentos_df = df

In [7]:
#divisões entre café da manhã e almoço em numero de Nutrientes, 1/5 para café e janta e 3/5 para almoço

proc = lista[0]/5
carc = lista[1]/5
gorc = lista[2]/5
vac = lista[3]/5
vcc = lista [4]/5
calc = lista[5]/5
ferc = lista[6]/5
cafc = 0

proa = lista[0]*3/5
cara = lista[1]*3/5
gora = lista[2]*3/5
vaa = lista[3]*3/5
vca = lista [4]*3/5
cala = lista[5]*3/5
fera = lista[6]*3/5
cafa = 1



In [8]:

# Definir o solver
solver = pywraplp.Solver.CreateSolver('GLOP')

# Variáveis de decisão para a quantidade de cada alimento
quantidades_alimentos = [solver.IntVar(0, solver.infinity(), alimentos_df.iloc[i]['d']) for i in range(len(alimentos_df))]

# Restrições para garantir que os requisitos nutricionais sejam atendidos
solver.Add(sum(alimentos_df.iloc[i]['c'] * quantidades_alimentos[i] for i in range(len(alimentos_df))) >= carc)
solver.Add(sum(alimentos_df.iloc[i]['p'] * quantidades_alimentos[i] for i in range(len(alimentos_df))) >= proc)
solver.Add(sum(alimentos_df.iloc[i]['g'] * quantidades_alimentos[i] for i in range(len(alimentos_df))) >= gorc)
solver.Add(sum(alimentos_df.iloc[i]['va'] * quantidades_alimentos[i] for i in range(len(alimentos_df))) >= vac)
solver.Add(sum(alimentos_df.iloc[i]['vc'] * quantidades_alimentos[i] for i in range(len(alimentos_df))) >= vcc)
solver.Add(sum(alimentos_df.iloc[i]['cal'] * quantidades_alimentos[i] for i in range(len(alimentos_df))) >= calc)
solver.Add(sum(alimentos_df.iloc[i]['f'] * quantidades_alimentos[i] for i in range(len(alimentos_df))) >= ferc)
solver.Add(sum(alimentos_df.iloc[i]['caf'] * quantidades_alimentos[i] for i in range(len(alimentos_df))) <= cafc)




# Função objetivo para minimizar o número total de alimentos
objective = solver.Objective()
for var in quantidades_alimentos:
    objective.SetCoefficient(var, 1)
objective.SetMinimization()

# Resolver o problema
status = solver.Solve()

# Verificar se a solução foi encontrada
if status == pywraplp.Solver.OPTIMAL:
    print('Café da manhã otimizado:')
    for i in range(len(alimentos_df)):
        if quantidades_alimentos[i].solution_value() > 0:
            print(f"{alimentos_df.iloc[i]['d']}: {quantidades_alimentos[i].solution_value()} porções")
else:
    print('Não foi possível encontrar uma solução ótima para a nova dieta.')

# Excluir os alimentos usados na dieta otimizada
alimentos_usados = []
for i in range(len(alimentos_df)):
    if quantidades_alimentos[i].solution_value() > 0:
        alimentos_usados.append(alimentos_df.iloc[i]['d'])

# Filtrar o DataFrame de alimentos para excluir os alimentos usados
alimentos_df_filtrado = alimentos_df[~alimentos_df['d'].isin(alimentos_usados)]
alimentos_df_filtrado = alimentos_df_filtrado[alimentos_df_filtrado['caf'] != 0]

# Atualizar o solver com o novo DataFrame de alimentos
solver = pywraplp.Solver.CreateSolver('GLOP')

# Variáveis de decisão para a quantidade de cada alimento
quantidades_alimentos = [solver.IntVar(0, solver.infinity(), alimentos_df_filtrado.iloc[i]['d']) for i in range(len(alimentos_df_filtrado))]

# Restrições para garantir que os requisitos nutricionais sejam atendidos
solver.Add(sum(alimentos_df_filtrado.iloc[i]['c'] * quantidades_alimentos[i] for i in range(len(alimentos_df_filtrado))) >= cara)
solver.Add(sum(alimentos_df_filtrado.iloc[i]['p'] * quantidades_alimentos[i] for i in range(len(alimentos_df_filtrado))) >= proa)
solver.Add(sum(alimentos_df_filtrado.iloc[i]['g'] * quantidades_alimentos[i] for i in range(len(alimentos_df_filtrado))) >= gora)
solver.Add(sum(alimentos_df_filtrado.iloc[i]['va'] * quantidades_alimentos[i] for i in range(len(alimentos_df_filtrado))) >= vaa)
solver.Add(sum(alimentos_df_filtrado.iloc[i]['vc'] * quantidades_alimentos[i] for i in range(len(alimentos_df_filtrado))) >= vca)
solver.Add(sum(alimentos_df_filtrado.iloc[i]['cal'] * quantidades_alimentos[i] for i in range(len(alimentos_df_filtrado))) >= cala)
solver.Add(sum(alimentos_df_filtrado.iloc[i]['f'] * quantidades_alimentos[i] for i in range(len(alimentos_df_filtrado))) >= fera)
solver.Add(sum(alimentos_df_filtrado.iloc[i]['caf'] * quantidades_alimentos[i] for i in range(len(alimentos_df_filtrado))) >= cafc)

# Função objetivo para minimizar o número total de alimentos
objective = solver.Objective()
for var in quantidades_alimentos:
    objective.SetCoefficient(var, 1)
objective.SetMinimization()

# Resolver o problema
status = solver.Solve()

# Verificar se a solução foi encontrada
print(75* '-')
if status == pywraplp.Solver.OPTIMAL:
    print('Almoço otimizado:')
    for i in range(len(alimentos_df_filtrado)):
        if quantidades_alimentos[i].solution_value() > 0:
            print(f"{alimentos_df_filtrado.iloc[i]['d']}: {quantidades_alimentos[i].solution_value()} porções")
else:
    print('Não foi possível encontrar uma solução ótima para a nova dieta.')

# Excluir os alimentos usados na dieta otimizada
alimentos_usados2 = []
for i in range(len(alimentos_df_filtrado)):
    if quantidades_alimentos[i].solution_value() > 0:
        alimentos_usados2.append(alimentos_df_filtrado.iloc[i]['d'])

# Filtrar o DataFrame de alimentos para excluir os alimentos usados
alimentos_df_filtrado2 = alimentos_df_filtrado[~alimentos_df_filtrado['d'].isin(alimentos_usados2)]

# Atualizar o solver com o novo DataFrame de alimentos
solver = pywraplp.Solver.CreateSolver('GLOP')

# Variáveis de decisão para a quantidade de cada alimento
quantidades_alimentos = [solver.IntVar(0, solver.infinity(), alimentos_df_filtrado2.iloc[i]['d']) for i in range(len(alimentos_df_filtrado2))]

# Restrições para garantir que os requisitos nutricionais sejam atendidos
solver.Add(sum(alimentos_df_filtrado2.iloc[i]['c'] * quantidades_alimentos[i] for i in range(len(alimentos_df_filtrado2))) >= carc)
solver.Add(sum(alimentos_df_filtrado2.iloc[i]['p'] * quantidades_alimentos[i] for i in range(len(alimentos_df_filtrado2))) >= proc)
solver.Add(sum(alimentos_df_filtrado2.iloc[i]['g'] * quantidades_alimentos[i] for i in range(len(alimentos_df_filtrado2))) >= gorc)
solver.Add(sum(alimentos_df_filtrado2.iloc[i]['va'] * quantidades_alimentos[i] for i in range(len(alimentos_df_filtrado2))) >= vac)
solver.Add(sum(alimentos_df_filtrado2.iloc[i]['vc'] * quantidades_alimentos[i] for i in range(len(alimentos_df_filtrado2))) >= vcc)
solver.Add(sum(alimentos_df_filtrado2.iloc[i]['cal'] * quantidades_alimentos[i] for i in range(len(alimentos_df_filtrado2))) >= calc)
solver.Add(sum(alimentos_df_filtrado2.iloc[i]['f'] * quantidades_alimentos[i] for i in range(len(alimentos_df_filtrado2))) >= ferc)
solver.Add(sum(alimentos_df_filtrado2.iloc[i]['caf'] * quantidades_alimentos[i] for i in range(len(alimentos_df_filtrado2))) >= cafc)

# Função objetivo para minimizar o número total de alimentos
objective = solver.Objective()
for var in quantidades_alimentos:
    objective.SetCoefficient(var, 1)
objective.SetMinimization()

# Resolver o problema
status = solver.Solve()

# Verificar se a solução foi encontrada
print(75* '-')
if status == pywraplp.Solver.OPTIMAL:
    print('Nova dieta otimizada encontrada:')
    for i in range(len(alimentos_df_filtrado2)):
        if quantidades_alimentos[i].solution_value() > 0:
            print(f"{alimentos_df_filtrado2.iloc[i]['d']}: {quantidades_alimentos[i].solution_value()} porções")
else:
    print('Não foi possível encontrar uma solução ótima para a nova dieta.')




Café da manhã otimizado:
Farinha, láctea, de cereais: 0.2620760872470564 porções
Achocolatado, pó: 0.01389266969151796 porções
Açúcar, cristal: 0.026931139212296857 porções
Gelatina, sabores variados, pó: 0.2689204241346054 porções
Amêndoa, torrada, salgada: 0.15426927250448622 porções
Gergelim, semente: 0.12604083792030815 porções
---------------------------------------------------------------------------
Almoço otimizado:
Creme de milho, pó: 0.8062443132998012 porções
Pipoca, com óleo de soja, sem sal: 0.5464187060301383 porções
Coentro, folhas desidratadas: 0.02353559370136414 porções
Carne, bovina, fígado, grelhado: 0.014409221902017292 porções
Chocolate, ao leite: 0.7296209501574829 porções
Soja, extrato solúvel, pó: 0.5015260958571395 porções
---------------------------------------------------------------------------
Nova dieta otimizada encontrada:
Farinha, de arroz, enriquecida: 0.0825727354842917 porções
Farinha, de mandioca, torrada: 0.2496349051975408 porções
Lambari, congel